In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix



In [3]:
import numpy as np
import pandas as pd

# get the features from the file features.txt
features = list()
with open(r"C:\Users\asus\OneDrive\Documents\PROJECTS\MINOR_2023_1st\UCI  HAR  Dataset\features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('No of Features: {}'.format(len(features)))

No of Features: 561


In [4]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

In [5]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

    
   # result = confusion_matrix(Y_true, Y_pred)

    #plt.figure(figsize=(10, 8))
   # sns.heatmap(result, 
    #            xticklabels= list(ACTIVITIES.values()), 
     #           yticklabels=list(ACTIVITIES.values()), 
      #          annot=True, fmt="d");
   # plt.title("Confusion matrix")
   # plt.ylabel('True label')
   # plt.xlabel('Predicted label')
    plt.show()

In [7]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [9]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = fr'C:\Users\asus\OneDrive\Documents\PROJECTS\MINOR_2023_1st\UCI  HAR  Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [10]:
# def load_y(subset):
#     """
#     The objective that we are trying to predict is a integer, from 1 to 6,
#     that represents a human activity. We return a binary representation of 
#     every sample objective as a 6 bits vector using One Hot Encoding
#     (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
#     """
#     filename = fr'C:\Users\asus\Downloads\Ho_gayaaaaaaaaaaaaa\UCI  HAR  Dataset/{subset}/y_{subset}.txt'
#     y = _read_csv(filename)[0]

#     return pd.get_dummies(y).to_numpy()

In [11]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [12]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [13]:
# Initializing parameters
epochs = 100
batch_size = 32
n_hidden = 32

In [14]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [ ]:
print("X_train : ",X_train.shape)
print("X_test : ",X_test.shape)
print("y_train : ",y_train.shape)
print("y_test : ",y_test.shape)

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

In [ ]:
# Define the soft attention mechanism layer
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, attention_dim):
        super(AttentionLayer, self).__init__()
        self.attention_dim = attention_dim
        self.W = tf.keras.layers.Dense(attention_dim, activation='tanh')
        self.u = tf.keras.layers.Dense(1, activation='softmax')

    def call(self, inputs):
        u_it = self.u(self.W(inputs))
        return tf.reduce_sum(inputs * u_it, axis=1)

In [ ]:
# Define the LSTM model with a soft attention mechanism layer
def create_model(X_train):
    attention_dim = 128
    lstm_units = 64

    input_layer = tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
    lstm_layer = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(input_layer)
    attention_layer = AttentionLayer(attention_dim)(lstm_layer)
    output_layer = tf.keras.layers.Dense(n_classes, activation='softmax')(attention_layer)

    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
y_train = pd.get_dummies(y_train.squeeze()).values
y_test = pd.get_dummies(y_test.squeeze()).values

In [ ]:
print(y_train.shape)
print(y_train[1])


print(y_test.shape)
y_test[0]

In [ ]:
from keras.callbacks import EarlyStopping

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

In [ ]:
# Train the model and evaluate on the test set
model = create_model(X_train)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),callbacks=[early_stopping])



In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


In [ ]:
from sklearn import metrics

predictions = model.predict(X_test)
class_labels = ['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']
max_test = np.argmax(y_test, axis=1)
max_predictions = np.argmax(predictions, axis=1)
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions)
sns.heatmap(confusion_matrix, xticklabels = class_labels, yticklabels = class_labels, annot = True, linewidths = 0.1, fmt='d', cmap = 'YlGnBu')
plt.title("Confusion matrix", fontsize = 15)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()